In [22]:
import pandas as pd
import os
import glob

In [25]:
def load_csvs(dir_car, dir_watch):
    """ load all data from csv files
    
    we assume that the directories contain only the desired files

    Parameters
    ----------
    dir : string
        

    Returns
    -------
    pandas DataFrame

    """
    print(os.getcwd())
    os.chdir(dir_car)
    files = [i for i in glob.glob('*_DataGroup_*.csv')]
    print(files)
    
    return None

In [26]:
# absolut base_dir
base_dir = '/Users/benjamin/code/starthack19/'
os.chdir(base_dir)
golden_table = load_csvs(os.path.join(base_dir, 'data/for_demo/car'), os.path.join(base_dir, 'data/for_demo/watch'))

/Users/benjamin/code/starthack19
[]
